In [1]:
# 참고: https://wikidocs.net/40710
import pandas as pd
data = pd.read_csv("fm_엑셀확인용.csv", error_bad_lines=False)
text = data[['제목', '본문', '댓글']]
text.head()

,제목,본문,댓글
0,비트코인 하락장으로 한 강간 디씨인,안 샀다고 한 닼,이제 자살이 아닌 타살이 될 것 오늘 펨코 비트코인 갤에서 연탄 자살 소동 일어나서...
1,킥킥이 사건 피해자 한울팍입니다 차가 해가 나날이 심해지네요 최군 발언 영상 포함,너무 스트레스 받아서 글이 더 어지러운 듯합니다 세줄 요약 난 신경 쓰기 싫은 데 ...,최군 왜 저럼 난 솔까 보다 킥킥이 저 사람이 더 악질 같은데 최소 는 대한민국 법...
2,요즘 사회가 진짜 존 나 각박해지긴 했다,예전엔 뭐 집안 어려운 여학생들 생리대 지원 이런 거 보면 아무 생각 안 들었는데 ...,그게 유리 바닥 부순다는 거몽키거프 남자들의 희생으로 겉으로만 화목해 보였던 거라고...
3,목마른 킹코브라에게 물을 줬더니 보인 놀라운 반응,NaN,막짤 저거 아니잖아 그냥 멀리 풀어줌 작성자 너무 하네이 분 좀 논란 있을 듯 여기...
4,이준석 오늘자 발언,태그를 지원하지 않는 브라우저입니다 비유가 너무 좋다 진짜 토론 잘하는 듯,이런 당연한 얘기를 다른 놈들은 하기가 힘든 건지 아니면 대가 리가 이상한 데로만 ...


In [2]:
#TF-IDF 행렬 생성
# TfidfVectorizer는 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(text['제목'])
X = vectorizer.fit_transform(text['본문'].values.astype('U'))
X = vectorizer.fit_transform(text['댓글'])
X.shape # TF-IDF 행렬의 크기 확인

(20, 1000)

In [3]:
# Topic Modeling: LDA 수행
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

In [4]:
lda_top=lda_model.fit_transform(X)

In [5]:
print(lda_model.components_)
print(lda_model.components_.shape)

[[0.79187796 0.76431081 0.90514979 ... 0.86039285 1.00037174 0.77915032]
 [0.68684755 0.80512995 0.7997033  ... 0.85834367 0.81415124 0.85011236]
 [0.78713649 1.02659003 0.80602361 ... 0.8100315  0.95699192 0.80078996]
 ...
 [0.93470531 0.82969104 0.82971003 ... 0.8880314  0.86147617 0.766606  ]
 [0.8298032  0.7799097  0.93013625 ... 0.88461828 1.01335476 0.78841542]
 [0.77100208 0.886344   0.86740813 ... 0.80313516 0.90979018 0.77886395]]
(10, 1000)


In [6]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('이유가', 1.23), ('수익', 1.14), ('바로', 1.13), ('전화로', 1.11), ('잠시', 1.07)]
Topic 2: [('문의', 1.13), ('때문에', 1.12), ('수가', 1.11), ('않음', 1.08), ('물론', 1.08)]
Topic 3: [('좋겠다', 1.11), ('스스로도', 1.06), ('당연한', 1.06), ('비키니', 1.05), ('뭐고', 1.05)]
Topic 4: [('메인', 1.11), ('멈춰', 1.08), ('않음', 1.05), ('페미들', 1.04), ('단체', 1.04)]
Topic 5: [('말로', 1.13), ('하지', 1.09), ('간부들', 1.06), ('없는데', 1.06), ('사실', 1.05)]
Topic 6: [('추신수', 1.2), ('많은', 1.08), ('맹유나', 1.08), ('그래도', 1.08), ('생각해보면', 1.07)]
Topic 7: [('솔직히', 1.16), ('그리고', 1.09), ('주자', 1.07), ('반포', 1.06), ('조만간', 1.05)]
Topic 8: [('직보리', 1.14), ('반댓말이', 1.11), ('제목만', 1.11), ('진짜', 1.11), ('저런', 1.1)]
Topic 9: [('여성', 1.13), ('밟고', 1.12), ('평생', 1.08), ('나오는', 1.06), ('새끼들아', 1.06)]
Topic 10: [('오히려', 1.16), ('하지', 1.11), ('엔딩', 1.09), ('나라는', 1.08), ('하면서', 1.07)]


In [ ]:
# 토큰화
# import nltk
# nltk.download('punkt')
# text['제목'] = text.apply(lambda row: nltk.word_tokenize(row['제목']), axis=1)
# text['본문'] = text.apply(lambda row: nltk.word_tokenize(str(row['본문']), axis=1)
# text['댓글'] = text.apply(lambda row: nltk.word_tokenize(row['댓글']), axis=1)
# text.head()

In [ ]:
# 토큰화 데이터 저장
# fm_dict_tokenized = {
#     '제목' : text['제목'],
#     '본문' : text['본문'],
#     '댓글' : text['댓글']
# }

# df = pd.DataFrame(fm_dict_tokenized) 
# df.to_csv('fm_토큰화.csv', index = False, encoding="utf-8-sig")

In [ ]:
# 한 글자 단어 삭제
# text['제목'] = text['제목'].apply(lambda x: [word for word in x if len(word) > 1])
# text['본문'] = text['본문'].apply(lambda x: [word for word in x if len(word) > 1])
# text['댓글'] = text['댓글'].apply(lambda x: [word for word in x if len(word) > 1])
# text.head()